In [1]:
import pandas as pd
import numpy as np
from functions import *
import random
import csv
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import GradientBoostingClassifier

In [2]:
#Load models
#REMINDER: We are going to need to use 'eval' to get the models usable
with open('models.csv', newline='') as f:
    reader = csv.reader(f)
    models = list(reader)
    
print(len(models))


###SELECT MODEL TO OPTIMIZE
model_num = 1

5


In [3]:
models

[['model_0',
  'model_1',
  'model_2',
  'model_3',
  'model_4',
  'model_06_13_20',
  'model_06_20_20',
  'model_6_27_20',
  'model_7_18_20',
  'model_7_25_20'],
 ["LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,\n                   intercept_scaling=1, l1_ratio=None, max_iter=50000,\n                   multi_class='auto', n_jobs=None, penalty='l2',\n                   random_state=75, solver='lbfgs', tol=5.993308915320019,\n                   verbose=0, warm_start=False)",
  'DecisionTreeClassifier(max_depth=33, max_leaf_nodes=546, random_state=75)',
  "RandomForestClassifier(criterion='entropy', max_depth=10, n_estimators=150,\n                       random_state=75)",
  'GradientBoostingClassifier(n_estimators=122, random_state=75)',
  'GaussianNB(var_smoothing=1e-12)',
  'LogisticRegression(max_iter=50000, random_state=75, tol=5.993308915320019)',
  'GaussianNB()',
  'GaussianNB()',
  'GradientBoostingClassifier(n_estimators=122, random_state=75)',
  '

In [4]:
df = pd.read_csv("../data/ufc-master.csv")

In [5]:
#Let's put all the labels in a dataframe
df['label'] = ''
#If the winner is not Red or Blue we can remove it.
mask = df['Winner'] == 'Red'
df['label'][mask] = 0
mask = df['Winner'] == 'Blue'
df['label'][mask] = 1

#df["Winner"] = df["Winner"].astype('category')
df = df[(df['Winner'] == 'Blue') | (df['Winner'] == 'Red')]


#Make sure lable is numeric
df['label'] = pd.to_numeric(df['label'], errors='coerce')

D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


In [6]:
#Let's fix the date
df['date'] = pd.to_datetime(df['date'])

In [7]:
#Create a label df:
label_df = df['label']

#Let's create an odds df too:
odds_df = df[['R_odds', 'B_odds']]

In [8]:
#Split the test set.  We are always(?) going to use the last 200 matches as the test set, so we don't want those around
#as we pick models

df_train = df[200:]
odds_train = odds_df[200:]
label_train = label_df[200:]

df_test = df[:200]
odds_test = odds_df[:200]
label_test = label_df[:200]

print(len(df_test))
print(len(odds_test))
print(len(label_test))

print(len(df_train))
print(len(odds_train))
print(len(label_train))

200
200
200
4126
4126
4126


In [9]:
display(df_train)
display(df_test)

,R_fighter,B_fighter,R_odds,B_odds,R_ev,B_ev,date,location,country,Winner,...,B_td_attempted_bout,R_td_pct_bout,B_td_pct_bout,R_sub_attempts_bout,B_sub_attempts_bout,R_pass_bout,B_pass_bout,R_rev_bout,B_rev_bout,label
200,Jake Matthews,Emil Meek,-200,170,50.000000,170.000000,2020-02-22,"Auckland, New Zealand",New Zealand,Red,...,1.0,0.57,0.00,0.0,0.0,3.0,0.0,0.0,0.0,0
201,Callan Potter,Song Kenan,170,-200,170.000000,50.000000,2020-02-22,"Auckland, New Zealand",New Zealand,Blue,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
202,Kai Kara France,Tyson Nam,-260,220,38.461538,220.000000,2020-02-22,"Auckland, New Zealand",New Zealand,Red,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
203,Loma Lookboonmee,Angela Hill,155,-175,155.000000,57.142857,2020-02-22,"Auckland, New Zealand",New Zealand,Blue,...,4.0,1.00,0.25,0.0,1.0,0.0,1.0,0.0,0.0,1
204,Priscila Cachoeira,Shana Dobson,180,-220,180.000000,45.454545,2020-02-22,"Auckland, New Zealand",New Zealand,Red,...,0.0,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4321,Duane Ludwig,Darren Elkins,-155,135,64.516129,135.000000,2010-03-21,"Broomfield, Colorado, USA",USA,Blue,...,2.0,0.00,0.50,0.0,0.0,0.0,0.0,0.0,0.0,1
4322,John Howard,Daniel Roberts,-210,175,47.619048,175.000000,2010-03-21,"Broomfield, Colorado, USA",USA,Red,...,1.0,1.00,1.00,0.0,1.0,0.0,1.0,0.0,1.0,0
4323,Brendan Schaub,Chase Gormley,-260,220,38.461538,220.000000,2010-03-21,"Broomfield, Colorado, USA",USA,Red,...,2.0,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0
4324,Mike Pierce,Julio Paulino,-420,335,23.809524,335.000000,2010-03-21,"Broomfield, Colorado, USA",USA,Red,...,0.0,0.60,0.00,0.0,0.0,6.0,0.0,0.0,0.0,0


,R_fighter,B_fighter,R_odds,B_odds,R_ev,B_ev,date,location,country,Winner,...,B_td_attempted_bout,R_td_pct_bout,B_td_pct_bout,R_sub_attempts_bout,B_sub_attempts_bout,R_pass_bout,B_pass_bout,R_rev_bout,B_rev_bout,label
0,Derrick Lewis,Aleksei Oleinik,-200,160,50.000000,160.000000,2020-08-08,"Las Vegas, Nevada, USA",USA,Red,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
1,Omari Akhmedov,Chris Weidman,115,-141,115.000000,70.921986,2020-08-08,"Las Vegas, Nevada, USA",USA,Blue,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
2,Maki Pitolo,Darren Stewart,125,-155,125.000000,64.516129,2020-08-08,"Las Vegas, Nevada, USA",USA,Blue,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
3,Yana Kunitskaya,Julija Stoliarenko,-190,150,52.631579,150.000000,2020-08-08,"Las Vegas, Nevada, USA",USA,Red,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
4,Beneil Dariush,Scott Holtzman,-200,160,50.000000,160.000000,2020-08-08,"Las Vegas, Nevada, USA",USA,Red,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,Karolina Kowalkiewicz,Yan Xiaonan,225,-265,225.000000,37.735849,2020-02-22,"Auckland, New Zealand",New Zealand,Blue,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
196,Ben Sosoli,Marcos Rogerio De Lima,130,-150,130.000000,66.666667,2020-02-22,"Auckland, New Zealand",New Zealand,Blue,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
197,Brad Riddell,Magomed Mustafaev,120,-140,120.000000,71.428571,2020-02-22,"Auckland, New Zealand",New Zealand,Red,...,11.0,1.0,0.72,0.0,0.0,2.0,0.0,0.0,0.0,0
198,Kevin Aguilar,Zubaira Tukhugov,-115,-105,86.956522,95.238095,2020-02-22,"Auckland, New Zealand",New Zealand,Blue,...,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,1


In [10]:
#Set a value for the nulls in the ranks

weightclass_list = ['B_match_weightclass_rank', 'R_match_weightclass_rank', "R_Women's Flyweight_rank", "R_Women's Featherweight_rank", "R_Women's Strawweight_rank", "R_Women's Bantamweight_rank", 'R_Heavyweight_rank', 'R_Light Heavyweight_rank', 'R_Middleweight_rank', 'R_Welterweight_rank', 'R_Lightweight_rank', 'R_Featherweight_rank', 'R_Bantamweight_rank', 'R_Flyweight_rank', 'R_Pound-for-Pound_rank', "B_Women's Flyweight_rank", "B_Women's Featherweight_rank", "B_Women's Strawweight_rank", "B_Women's Bantamweight_rank", 'B_Heavyweight_rank', 'B_Light Heavyweight_rank', 'B_Middleweight_rank', 'B_Welterweight_rank', 'B_Lightweight_rank', 'B_Featherweight_rank', 'B_Bantamweight_rank', 'B_Flyweight_rank', 'B_Pound-for-Pound_rank']
df_train[weightclass_list] = df_train[weightclass_list].fillna(17)
df_test[weightclass_list] = df_test[weightclass_list].fillna(17)


D:\Anaconda3\lib\site-packages\pandas\core\frame.py:2963: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


In [11]:
df_test
#df_test.to_csv('test.csv')

,R_fighter,B_fighter,R_odds,B_odds,R_ev,B_ev,date,location,country,Winner,...,B_td_attempted_bout,R_td_pct_bout,B_td_pct_bout,R_sub_attempts_bout,B_sub_attempts_bout,R_pass_bout,B_pass_bout,R_rev_bout,B_rev_bout,label
0,Derrick Lewis,Aleksei Oleinik,-200,160,50.000000,160.000000,2020-08-08,"Las Vegas, Nevada, USA",USA,Red,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
1,Omari Akhmedov,Chris Weidman,115,-141,115.000000,70.921986,2020-08-08,"Las Vegas, Nevada, USA",USA,Blue,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
2,Maki Pitolo,Darren Stewart,125,-155,125.000000,64.516129,2020-08-08,"Las Vegas, Nevada, USA",USA,Blue,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
3,Yana Kunitskaya,Julija Stoliarenko,-190,150,52.631579,150.000000,2020-08-08,"Las Vegas, Nevada, USA",USA,Red,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
4,Beneil Dariush,Scott Holtzman,-200,160,50.000000,160.000000,2020-08-08,"Las Vegas, Nevada, USA",USA,Red,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,Karolina Kowalkiewicz,Yan Xiaonan,225,-265,225.000000,37.735849,2020-02-22,"Auckland, New Zealand",New Zealand,Blue,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
196,Ben Sosoli,Marcos Rogerio De Lima,130,-150,130.000000,66.666667,2020-02-22,"Auckland, New Zealand",New Zealand,Blue,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
197,Brad Riddell,Magomed Mustafaev,120,-140,120.000000,71.428571,2020-02-22,"Auckland, New Zealand",New Zealand,Red,...,11.0,1.0,0.72,0.0,0.0,2.0,0.0,0.0,0.0,0
198,Kevin Aguilar,Zubaira Tukhugov,-115,-105,86.956522,95.238095,2020-02-22,"Auckland, New Zealand",New Zealand,Blue,...,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,1


In [12]:
#1. Set features
#2. Set Hyperparameters
#3. Set EV
#4. Remove Features

In [13]:
models[1][model_num]

'DecisionTreeClassifier(max_depth=33, max_leaf_nodes=546, random_state=75)'

In [14]:
test_model_name = models[0][model_num]
test_model = eval(models[1][model_num])
test_model_features = eval(models[2][model_num])
test_model_ev = eval(models[3][model_num])

In [15]:
old_test_model = test_model
old_test_model_features = test_model_features
old_test_model_ev = test_model_ev

In [16]:
print(df_test.columns.tolist()) #Keeps us from truncating

['R_fighter', 'B_fighter', 'R_odds', 'B_odds', 'R_ev', 'B_ev', 'date', 'location', 'country', 'Winner', 'title_bout', 'weight_class', 'gender', 'no_of_rounds', 'B_current_lose_streak', 'B_current_win_streak', 'B_draw', 'B_avg_SIG_STR_landed', 'B_avg_SIG_STR_pct', 'B_avg_SUB_ATT', 'B_avg_TD_landed', 'B_avg_TD_pct', 'B_longest_win_streak', 'B_losses', 'B_total_rounds_fought', 'B_total_title_bouts', 'B_win_by_Decision_Majority', 'B_win_by_Decision_Split', 'B_win_by_Decision_Unanimous', 'B_win_by_KO/TKO', 'B_win_by_Submission', 'B_win_by_TKO_Doctor_Stoppage', 'B_wins', 'B_Stance', 'B_Height_cms', 'B_Reach_cms', 'B_Weight_lbs', 'R_current_lose_streak', 'R_current_win_streak', 'R_draw', 'R_avg_SIG_STR_landed', 'R_avg_SIG_STR_pct', 'R_avg_SUB_ATT', 'R_avg_TD_landed', 'R_avg_TD_pct', 'R_longest_win_streak', 'R_losses', 'R_total_rounds_fought', 'R_total_title_bouts', 'R_win_by_Decision_Majority', 'R_win_by_Decision_Split', 'R_win_by_Decision_Unanimous', 'R_win_by_KO/TKO', 'R_win_by_Submission',

In [17]:
#1. set features
my_pos_features = ['R_fighter', 'B_fighter', 'R_odds', 'B_odds', 'R_ev', 'B_ev',
       'location', 'country', 'title_bout', 'weight_class', 'gender',
       'no_of_rounds', 'B_current_lose_streak', 'B_current_win_streak',
       'B_draw', 'B_avg_SIG_STR_landed', 'B_avg_SIG_STR_pct', 'B_avg_SUB_ATT',
       'B_avg_TD_landed', 'B_avg_TD_pct', 'B_longest_win_streak', 'B_losses',
       'B_total_rounds_fought', 'B_total_title_bouts',
       'B_win_by_Decision_Majority', 'B_win_by_Decision_Split',
       'B_win_by_Decision_Unanimous', 'B_win_by_KO/TKO', 'B_win_by_Submission',
       'B_win_by_TKO_Doctor_Stoppage', 'B_wins', 'B_Stance', 'B_Height_cms',
       'B_Reach_cms', 'B_Weight_lbs', 'R_current_lose_streak',
       'R_current_win_streak', 'R_draw', 'R_avg_SIG_STR_landed',
       'R_avg_SIG_STR_pct', 'R_avg_SUB_ATT', 'R_avg_TD_landed', 'R_avg_TD_pct',
       'R_longest_win_streak', 'R_losses', 'R_total_rounds_fought',
       'R_total_title_bouts', 'R_win_by_Decision_Majority',
       'R_win_by_Decision_Split', 'R_win_by_Decision_Unanimous',
       'R_win_by_KO/TKO', 'R_win_by_Submission',
       'R_win_by_TKO_Doctor_Stoppage', 'R_wins', 'R_Stance', 'R_Height_cms',
       'R_Reach_cms', 'R_Weight_lbs', 'R_age', 'B_age', 'lose_streak_dif',
       'win_streak_dif', 'longest_win_streak_dif', 'win_dif', 'loss_dif',
       'total_round_dif', 'total_title_bout_dif', 'ko_dif', 'sub_dif',
       'height_dif', 'reach_dif', 'age_dif', 'sig_str_dif', 'avg_sub_att_dif',
       'avg_td_dif', 'empty_arena', 'B_match_weightclass_rank', 'R_match_weightclass_rank', 
        "R_Women's Flyweight_rank", "R_Women's Featherweight_rank", "R_Women's Strawweight_rank",
        "R_Women's Bantamweight_rank", 'R_Heavyweight_rank', 'R_Light Heavyweight_rank', 
        'R_Middleweight_rank', 'R_Welterweight_rank', 'R_Lightweight_rank', 'R_Featherweight_rank', 
        'R_Bantamweight_rank', 'R_Flyweight_rank', 'R_Pound-for-Pound_rank', "B_Women's Flyweight_rank", 
        "B_Women's Featherweight_rank", "B_Women's Strawweight_rank", "B_Women's Bantamweight_rank", 
        'B_Heavyweight_rank', 'B_Light Heavyweight_rank', 'B_Middleweight_rank', 'B_Welterweight_rank', 
        'B_Lightweight_rank', 'B_Featherweight_rank', 'B_Bantamweight_rank', 'B_Flyweight_rank', 
        'B_Pound-for-Pound_rank', 'better_rank']

In [18]:
print(test_model_name)
print(test_model)
print(test_model_features)
print(test_model_ev)



model_1
DecisionTreeClassifier(max_depth=33, max_leaf_nodes=546, random_state=75)
['B_Light Heavyweight_rank', "B_Women's Featherweight_rank", "B_Women's Flyweight_rank", 'R_win_by_Decision_Majority', 'R_win_by_Decision_Unanimous', "R_Women's Flyweight_rank", 'R_draw', 'B_age', 'R_avg_TD_pct', 'empty_arena']
10


In [19]:
def save_model():
    score = evaluate_model(test_model, test_model_features, test_model_ev, df_train, label_train, odds_train, df_test, label_test,
                         odds_test, verbose = True)
    models[0][model_num] =  test_model_name 
    models[1][model_num] = test_model
    models[2][model_num] = test_model_features
    models[3][model_num] = test_model_ev
    models[4][model_num] = score    
    
    with open('models.csv', 'w', newline='') as outfile:
        writer = csv.writer(outfile)
        for row in models:
            print("HI")
            writer.writerow(row)

    outfile.close()    

In [20]:
def print_model():
    print()
    print(test_model_name)
    print(test_model)
    print(test_model_features)
    print(test_model_ev)
    print()

In [ ]:
keep_going = True
while(keep_going):
    #1. Set Features
    #get_best_features(pos_features, m, df, cur_features, labels, odds, scale=False)
    test_model_features = (get_best_features(my_pos_features, test_model, df_train, test_model_features, label_train, odds_train, 
                                             min_ev=test_model_ev))
    print_model()
    save_model()
    #2 Set hyperparameters
    #def tune_hyperparameters(input_model, input_features, input_df, input_labels, odds_input):
    test_model = tune_hyperparameters(test_model, test_model_features, df_train, label_train, odds_train, 
                                      min_ev=test_model_ev)
    print_model()
    save_model()
    #3. Set EV
    #def tune_ev(input_model, input_features, input_df, input_labels, odds_input, verbose=False):
    test_model_ev = tune_ev(test_model, test_model_features, df_train, label_train, odds_train, verbose=False)

    old_test_model_features = test_model_features #This prevents
                                                  #an uneccesary loop
    print_model()
    save_model()
    #4. Remove Features
    #def remove_to_improve(cur_features, m, df, labels, odds, scale=False, min_ev = 0):
    test_model_features = remove_to_improve(test_model_features, test_model, df_train, label_train, odds_train, min_ev = test_model_ev)    
    keep_going = False
    if old_test_model != test_model:
        print("The hyperparameters are different")
        print("OLD:")
        print(old_test_model)
        print("NEW:")
        print(test_model)
        keep_going = True
        old_test_model = test_model
    if old_test_model_features != test_model_features:
        print("The features are different")
        print("OLD:")
        print(old_test_model_features)
        print("NEW:")
        print(test_model_features)
        keep_going = True
        old_test_model_features = test_model_features
    if old_test_model_ev != test_model_ev:
        print("The EV is different")
        print("OLD:")
        print(old_test_model_ev)
        print("NEW:")
        print(test_model_ev)
        keep_going = True
        old_test_model_ev = test_model_ev
    print_model()
    save_model()
    

Current best score is: -2881.5631646077436
Feature: B_avg_TD_landed Score: 4018.519213566461
Feature: win_dif Score: 5897.153719678602
The best feature was win_dif.  It scored 5897.153719678602
Current best score is: 5897.153719678602
Feature: title_bout Score: 8257.34979446948
Feature: R_Lightweight_rank Score: 9471.876659187665
The best feature was R_Lightweight_rank.  It scored 9471.876659187665
Current best score is: 9471.876659187665
Feature: B_win_by_Decision_Majority Score: 9602.065477102
Feature: R_Heavyweight_rank Score: 13518.30294566611
The best feature was R_Heavyweight_rank.  It scored 13518.30294566611
Current best score is: 13518.30294566611
Feature: B_draw Score: 14139.072058793987
Feature: B_Heavyweight_rank Score: 14899.463158250845
The best feature was B_Heavyweight_rank.  It scored 14899.463158250845
Current best score is: 14899.463158250845
Feature: B_Pound-for-Pound_rank Score: 17295.92365862155
The best feature was B_Pound-for-Pound_rank.  It scored 17295.9236586

(3759, 16)

(3759,)

(3759, 2)

(200, 16)

...

(200,)

(200, 2)

,t1_odds,t2_odds,t1_prob,t2_prob,winner
0,-200,160,1.000000,0.000000,0
1,115,-141,0.947368,0.052632,1
2,125,-155,0.900000,0.100000,1
3,-190,150,0.857143,0.142857,0
4,-200,160,0.875000,0.125000,0
...,...,...,...,...,...
195,225,-265,1.000000,0.000000,1
196,130,-150,1.000000,0.000000,1
197,120,-140,0.539130,0.460870,0
198,-115,-105,1.000000,0.000000,1



          Number of matches: 200
          Number of bets: 176
          Number of winning bets: 93
          Number of losing bets: 83
          Number of underdog bets: 80
          Number of underdog wins: 29
          Number of underdog losses: 51
          Number of Favorite bets: 88
          Number of favorite wins: 60
          Number of favorite losses: 28
          Number of even bets: 8
          Number of even wins: 4
          Number of even losses: 4
          Profit: -202.4331635319566
          Profit per bet: -1.1501884291588444
          Profit per match: -1.012165817659783
          
          
HI
HI
HI
HI
HI


Starting New Run for DecisionTree


Previous Best Score: 19172.06547724608
Criterion: gini splitter: random max_depth: 31 min_samples_leaf: 1 max_leaf_nodes: 565 -17463.93610660401
Criterion: gini splitter: best max_depth: 31 min_samples_leaf: 1 max_leaf_nodes: 565 19172.06547724608
Criterion: entropy splitter: random max_depth: 31 min_samples_leaf: 1 max_lea

Criterion: entropy splitter: best max_depth: 30 min_samples_leaf: 2 max_leaf_nodes: 564 11287.329682283746
Criterion: gini splitter: random max_depth: 32 min_samples_leaf: 2 max_leaf_nodes: 564 -8658.243225502392
Criterion: gini splitter: best max_depth: 32 min_samples_leaf: 2 max_leaf_nodes: 564 15331.6734117755
Criterion: entropy splitter: random max_depth: 32 min_samples_leaf: 2 max_leaf_nodes: 564 -10735.263840525758
Criterion: entropy splitter: best max_depth: 32 min_samples_leaf: 2 max_leaf_nodes: 564 12557.71327777391
Criterion: gini splitter: random max_depth: 34 min_samples_leaf: 2 max_leaf_nodes: 564 -9247.256374209937
Criterion: gini splitter: best max_depth: 34 min_samples_leaf: 2 max_leaf_nodes: 564 15583.514404762987
Criterion: entropy splitter: random max_depth: 34 min_samples_leaf: 2 max_leaf_nodes: 564 -14158.80335099516
Criterion: entropy splitter: best max_depth: 34 min_samples_leaf: 2 max_leaf_nodes: 564 11353.210940644753
Criterion: gini splitter: random max_depth:

In [ ]:
#Get new score:
#def evaluate_model(input_model, input_features, input_ev, train_df, train_labels, train_odds, test_df, test_labels, test_odds, verbose=True):
score = evaluate_model(test_model, test_model_features, test_model_ev, df_train, label_train, odds_train, df_test, label_test,
                         odds_test, verbose = True)

print(test_model_name) 
print(models[0][model_num])
print()
print(test_model)
print(eval(models[1][model_num]))
print()
print(test_model_features) 
print(eval(models[2][model_num]))
print()
print(test_model_ev)
print(eval(models[3][model_num]))

In [ ]:
models[0][model_num] =  test_model_name 
models[1][model_num] = test_model
models[2][model_num] = test_model_features
models[3][model_num] = test_model_ev
models[4][model_num] = score

In [ ]:
with open('models.csv', 'w', newline='') as outfile:
    writer = csv.writer(outfile)
    for row in models:
        print("HI")
        writer.writerow(row)
    
outfile.close()

In [ ]:
with open('models.csv', newline='') as f:
    reader = csv.reader(f)
    data = list(reader)
    
print(len(data))

In [ ]:
data